In [12]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from checkpoint_callback import CheckpointCallback

In [11]:
%load_ext autoreload
%autoreload 2

In [19]:
MODEL_NAME = "dummy"

In [2]:
# Create the environment
env = gym.make("CartPole-v1")

In [16]:
# Define the checkpoint callback
checkpoint_callback = CheckpointCallback(save_freq=500, save_path="./checkpoints/", model_name=MODEL_NAME)

# Initialize the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the PPO model with the callback
model.learn(total_timesteps=1000, callback=checkpoint_callback)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Checkpoint saved: ./checkpoints/dummy_checkpoint_500.zip
Checkpoint saved: ./checkpoints/dummy_checkpoint_1000.zip
Checkpoint saved: ./checkpoints/dummy_checkpoint_1500.zip
Checkpoint saved: ./checkpoints/dummy_checkpoint_2000.zip
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.7     |
|    ep_rew_mean     | 23.7     |
| time/              |          |
|    fps             | 2317     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------


In [ ]:
# Evaluate the model (using the latest checkpoint for testing purposes)
latest_checkpoint = f"./checkpoints/{MODEL_NAME}_checkpoint_100.zip"
loaded_model = PPO.load(latest_checkpoint, env=env)

# Evaluate the model
mean_reward, std_reward = evaluate_policy(loaded_model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward}")


/home/hice1/rmehta98/.conda/envs/rl_env/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 496.4 +/- 10.8
